# Classification with Bayesian methods

Last revision: Mon 08 Jun 2024 by Peiyu Tang

## Packages

|   Package      | Version     |
| -----------    | ----------- |
| `matplotlib`   | `3.9.0`     |
| `numpy`        | `1.26.4`    |
| `pandas`       | `2.2.2`     |
| `scikit-learn` | `1.5.0`     |

## Introduction

We start with a quick refresher on what a classification problem is.

### The classification problem
For a classification problem, for some population $X$ and a finite set of labels $Y$, we are interested in fitting/finding mapping function $h: X \to Y$ that matches some relation between $X$ and $Y$. In the case where $Y = \{0, 1\}$, we say that the problem is a binary classification problem. 

Particularly for this lab,
1. We begin by looking at the inner workings of Naive Bayes on binary classification problems and provide an implementation for it.   
2. We then take a look at the case of multinomial classification for Naive Bayes and also enable smoothing for our implementation. 

The dataset we will be using is the real data set on the sinking of the Titanic. For a description of the variables and more information on the data please read [here](https://www.kaggle.com/c/titanic-gettingStarted/data). 

## Quick recap on Naive Bayes

Naive Bayes is a straightforward model that leverages Bayes's rule to find the most probable prediction based solely on historical data. Naive Bayes has very few assumptions about the data and it is general enough to apply to most problems. In most cases, Naive Bayes is used as the baseline model for complex problems, hence any more complicated model should do at least better than Naive Bayes! 

We start with the famous Bayes rule, which tells us that for 2 random variables $X, Y$ we always have that 
$$
    P(y | x) = \frac{P(x | y)P(y)}{P(x)}
$$
which 
* $P(y|x)$ is known as the **posterior**,
* $P(x | y)$ is known as **likelihood**,
* $P(y)$ is known as the **prior** distribution and
* $P(x)$ is the **normalization factor**.

Then for a set of historical data $x_1, x_2, \dots, x_n$, we can apply a few tricks to make our lives a bit easier.
1. With especially classification problems, we are only comparing the probability of different classes, therefore the marginal distribution $P(X)$ can be ignored. 
2. In Bayesian statistics, we usually write likelihood as $L(x_1, x_2, \dots, x_n|y)$ and since we can assume that each furthermore,
3. In most situations, we assume that the sampled random variable is i.i.d hence
$$
    L(x_1, \dots, x_n | y) = \prod_{i = 1}^n P(x_i | y) \implies  P(y|x_1, \dots, x_n) \propto P(y)\prod_{i = 1}^n P(x_i | y).
$$
Then for all classes $c_i \in Y$, we wish to find 
$$
    c^* = \underset{c_1, \dots, c_k \in Y}{\text{argmax}} P(y|x_1, \dots, x_n)
$$

which we call $c^*$ the maximum likelihood hypothesis. Also, note here that $P(x_i | y)$ can be computed from our historical data.

### Discriminative vs Generative modelling
Whenever we consider using probabilistic models in our problem-solving, we usually face a decision to use discriminative or generative models.

In **discriminative modelling**, we are only interested in modelling the conditional relationship between the random variables that we need, i.e. the posterior distribution $P(y|x)$.

In **generative modelling**, we are interested in modelling the joint distribution $P(x, y)$ which we can then sample from to obtain new data.

Suppose that there are $C$ classes for classification label $Y$ and $K$ different possible values for each $x_i$. For generative modelling, referencing the law of marginal distribution we have that
$$
    P(x, y) = P(y | x) P(x) \quad \text{with} \quad P(x) = \sum_{i = 1}^C P(y = c_i)P(x_1, \dots, x_n | y = c_i)
$$
Therefore we still need to compute the marginal factor $P(x)$.


### Implementation

We now have a look at the `titanic` dataset, which tracks the survivalibility guests on the famous sinked ship Titanic. We start by loading the dataset into a dataframe.




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


titanic_train = pd.read_csv("./titanic_train.csv")
titanic_test = pd.read_csv("./titanic_test.csv")

feature_set = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]

# selecting columns
X_train = titanic_train[feature_set]
X_test = titanic_test[feature_set]
y_train = titanic_train["Survived"]
y_test = titanic_test["Survived"]


print(f"Shape of training set: {titanic_train.shape}")
print(f"Shape of test set: {titanic_train.shape}")
X_train

<font color='purple'>
    
#### Exercise 1: 
Use the template given below or otherwise, implement the Naive Bayes to find the maximum likelihood hypothesis for the test set `titanic_test` using the given features. Comment on the accuracy of your model.

In [ ]:
from functools import reduce

class Naive_Bayes():
    def __init__(self) -> None:
        pass

    def fit(self, X_train, y_train):
        '''
        What is actually required at this step? ;), do we really need to do anything?
        '''
        return self

    def predict(self, X_test):
        '''
        1. Find a way to compute the number of classes from y_train, you might find np.unique(.) useful
        2. Prior of P(y) should also be computed from y_train as well
        3. For each row in X_test, we will need to compute class with the maximum likelihood
            * maybe create another function for this, use the vectorized arr[: index] == value to filter
                X_train to compute the probabilities
            * you might also find np.prod(.) very helpful
            * you may also find np.max very useful here as well
        '''
        pass
    def benmark(self, X_test, y_test):
        '''
        Use your self.predict(.) function to obtain the predicted classes, then use check the result element
        by element by using the arr[: index] == value vectorized operator again. Then divide it with the
        the length of X_test for the accuracy score.
        '''
        pass

nb_model = Naive_Bayes().fit(
    X_train.to_numpy(),
    y_train.to_numpy()
)
results = nb_model.predict(X_test.to_numpy())
accuracy = nb_model.benchmark(X_test.to_numpy(), y_test)
print(f"Test accuracy: {accuracy}")

## Laplace Smoothing

From our implementation above, we see that in cases when we get $P(x_i | y) = 0$, our entire likelihood inference will decay to 0. Therefore to combat against such instances, we can use a technique called Laplace smoothing. In Laplace smoothing, instead of directly using the data to compute the conditional probabilities, we use the following
$$
 P_L(x_i | y) = \frac{P(x_i | y) + \alpha}{N + \alpha n}
$$
where 
* $\alpha$: a user-chosen hyper-parameter, 
* $n$: the number of features (like before) and 
* $N$: the number of data instances. 

In this way, when $P(x_i | y) = 0$ we get that 
$$
 P_L(x_i | y) = \frac{\alpha}{N + \alpha n} > 0.
$$

## Laplace Smoothing

From our implementation above, we see that in cases when we get $P(x_i | y) = 0$, our entire likelihood inference will decay to 0. Therefore to combat against such instances, we can use a technique called Laplace smoothing. In Laplace smoothing, instead of directly using the data to compute the conditional probabilities, we use the following
$$
 P_L(x_i | y) = \frac{P(x_i | y) + \alpha}{N + \alpha n}
$$
where 
* $\alpha$: a user-chosen hyper-parameter, 
* $n$: the number of features (like before) and 
* $N$: the number of data instances. 

In this way, when $P(x_i | y) = 0$ we get that 
$$
 P_L(x_i | y) = \frac{\alpha}{N + \alpha n} > 0.
$$

<font color='purple'>
    
#### Exercise 2: 
Basing on your previous implementation (or otherwise), implement 
1. Laplace smoothing and 
2. customizable prior values

for your Naive Bayes estimator. Then use $\alpha = 3$ and a custom prior distribution to compute your results again.

In [ ]:
class Naive_Bayes():
    '''
    You can copy your code from the previous section, think about which function you need to actually change.
    '''
    def __init__(self) -> None:
        pass

    def fit(self, X_train, y_train):
        return self

    def predict(self, X_test, alpha = 0, prior = None):
        '''
        Does the processing actually change, maybe all it needs just a change in formula or parameters that we use. ;)
        '''
        pass
    def benmark(self, X_test, y_test):
        pass

In [ ]:
nb_model = Naive_Bayes().fit(
    X_train.to_numpy(),
    y_train.to_numpy()
)
results = nb_model.predict(X_test.to_numpy(), alpha = 3)
accuracy = nb_model.benchmark(X_test.to_numpy(), y_test, alpha = 3)
print(f"Test accuracy: {accuracy}")

In [ ]:
num0 = np.sum([1 * (d[0] == 0 and d[1] == 0) for d in results])
print(f"Number of predictions with value of 0: {num0}/{len(results)} instances")

## Extension: Continuous Naive Bayes: LDA and QDA

Naive Bayes can also be used to handle continuous features, those models is more commonly known as Linear or Quadratic Discriminant Analysis (LDA or QDA). In LDA,the model is rather straightforward, we start with assuming our likelihood distribution is normally distributed, i.e.
$$
    P(x | y = c_i) \sim \mathcal{N}(\mu_i, \Sigma)
$$
with each different class $c_i$ has a different mean with the same variance $\Sigma$. We then pick the class $c_i$ that maximizes this the posterior distribution. This is hence equivalent to finding the maximum for the log of posterior
$$
\ln P(x | y = c_i) = -\frac{1}{2} (x - \mu_{c_i})^T \Sigma^{-1} (x - \mu_{c_i}) + \ln P(y = c_i) + C
$$
which 
1. $\mu_{c_i}$ is the mean for each class which are all assumed to be different and
2. $C$ is a constant term from the normal distribution and the normalizing factor $P(x)$.

For QDA, the expression changes to 
$$
\ln P(x | y = c_i) = -\frac{1}{2} \ln|\Sigma_{c_i}| -\frac{1}{2} (x - \mu_{c_i})^T \Sigma_{c_i}^{-1} (x - \mu_{c_i}) + \ln P(y = c_i) + C
$$
since each $\Sigma_{c_i}$ of different classes are no longer assumed to be the same.
We will here omit the derivation as it is beyond the scope of this course, but you have a read about it [here](https://en.wikipedia.org/wiki/Linear_discriminant_analysis). 

We now look into how to apply LDA/QDA directly using the `GaussianNB` class from `sklearn`. The data below is generated by a ```Gaussian mixture model```. For each class there is a separate 2-dimensional Gaussian distribution over the features `x1`, `x2`. 

In [ ]:
from scipy.stats import bernoulli
from scipy.stats import multivariate_normal

class GaussianMixture:
    def __init__(self,mean0,cov0,mean1,cov1):
        """ construct a mixture of two gaussians. mean0 is 2x1 vector
            of means for class 0, cov0 is 2x2 covariance matrix for class 0.
        Similarly for class 1
        """

        self.mean0 = mean0
        self.mean1 = mean1
        self.cov0 = cov0
        self.cov1 = cov1
        self.rv0 = multivariate_normal(mean0, cov0)
        self.rv1 = multivariate_normal(mean1, cov1)

    def plot(self,data=None):
        x1 = np.linspace(-4,4,100)
        x2 = np.linspace(-4,4,100)
        X1,X2 = np.meshgrid(x1,x2)
        pos = np.empty(X1.shape+(2,))
        pos[:,:,0] = X1
        pos[:,:,1]= X2
        a = self.rv1.pdf(pos)/self.rv0.pdf(pos)

        if data:
            nplots = 4
        else:
            nplots = 3

        fig, ax = plt.subplots(1,nplots,figsize = (5*nplots,5))
        [ax[i].spines['left'].set_position('zero') for i in range(0,nplots)]
        [ax[i].spines['right'].set_color('none') for i in range(0,nplots)]
        [ax[i].spines['bottom'].set_position('zero') for i in range(0,nplots)]
        [ax[i].spines['top'].set_color('none') for i in range(0,nplots)]

        ax[0].set_title("p(x1,x2|y = 1")
        ax[1].set_title("p(x1,x2|y = 0")
        ax[2].set_title("P(y = 1|x1,x2)")
        [ax[i].set_xlim([-4,4]) for i in range(0,3)]
        [ax[i].set_ylim([-4,4]) for i in range(0,3)]

        cn = ax[0].contourf(x1,x2,self.rv1.pdf(pos))
        cn2 = ax[1].contourf(x1,x2,self.rv0.pdf(pos))
        z = a/(1.0+a)
        cn3 = ax[2].contourf(x1,x2,z)
        ct = ax[2].contour(cn3,levels=[0.5])
        ax[2].clabel(ct)


        if data:
            X, Y = data
            colors = ["blue" if target < 1 else "red" for target in Y]
            colors = np.array(colors)
            x = X[:,0]
            y = X[:,1]
            # import IPython; IPython.embed()
            yis1 = np.where(Y==1)[0]
            yis0 = np.where(Y!=1)[0]
            ax[3].set_title("Samples colored by class")
            ax[3].scatter(x,y,s=30,c=colors,alpha=.5)
            ax[0].scatter(x[yis1],y[yis1],s=5,c=colors[yis1],alpha=.3)
            ax[1].scatter(x[yis0],y[yis0],s=5,c=colors[yis0],alpha=.3)
            ax[2].scatter(x,y,s=5,c=colors,alpha=.3)
        plt.show()

    def sample(self,n_samples,py,plot=False):
        """
        samples Y according to py and corresponding features x1,x2
        according to the gaussian for the corresponding class
        """
        Y = bernoulli.rvs(py,size=n_samples)
        X = np.zeros((n_samples,2))
        for i in range(n_samples):
            if Y[i] == 1:
                X[i,:] = self.rv1.rvs()
            else:
                X[i,:] = self.rv0.rvs()
        if plot:
            self.plot(data=(X,Y))
        return X,Y

In [ ]:
# Generates some data from a Gaussian Mixture Model.
mean0 = [-1,-1]  # the mean of the gaussian for class 0
mean1 = [1,1] # the mean of the gaussian for class 1
cov0 = [[.5, .28], [.28, .5]] # the covariance matrix for class 0
cov1 = [[1, -.8], [-.8, 1]] # the covariance matrix for class 1
mixture = GaussianMixture(mean0,cov0,mean1,cov1)
mX,mY = mixture.sample(500,0.5,plot=True)

<font color='purple'>
    
#### Exercise 3: 
Fit a Gaussian Naive Bayes model using the `GaussianNB` from the `sklearn.naive_bayes` library

In [ ]:
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt

## start coding here...